<a href="https://colab.research.google.com/github/yskuchi/wf_denoising/blob/master/denoising1D_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waveform denoising 'denoising1D_tf2'
Author: Yusuke Uchiyama

A denoising convolutional autoencoder with Tensorflow2.x
applied to **a set of or single** waveform data using 1D convolutional autoencoder with multiple input channels.  
See [Bitbucket repository](https://bitbucket.org/meg_ilc_tokyo/wf_denoising/src/master/) or 
[GitHub repository](https://github.com/yskuchi/wf_denoising)

Noise from data is added to MC signal data.
You need datasets of signal and noise, separately, in pickle format.

## Environment
As of 2022 Aug, tested with the following:

* Google Colab
* CPU, GPU, or TPU (experimental)
* Python 3.7
* TensorFlow 2.8.2
* Comet ML 3.31


Note: If you are running this in a colab notebook, we recommend you enable a free GPU by going:
> Runtime   →   Change runtime type   →   Hardware Accelerator: GPU

## Setting

### Comet ML

In [ ]:
! pip install typing-extensions comet-ml
! pip install typing-extensions
#! [ ! -z "$COLAB_GPU" ] && pip install typing-extensions==3.7.4 comet-ml

In [ ]:
# import comet_ml in the top of your file
from comet_ml import Experiment

### Other packages

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import json

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Concatenate
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

print ('Python version: ' + str(sys.version_info))
print ('TensorFlow version: ' + str(tf.__version__))

### GPU
To use GPU on Google colab, specify GPU in runtime type. 

In [ ]:
# check GPU
tf.test.gpu_device_name()
!echo $COLAB_GPU

### TPU
To use TPU on Google colab, it is not enough to specify TPU in runtime type.
See "Setup TPU".

In [ ]:
# check TPU
!echo $COLAB_TPU_ADDR

### Parameters

In [ ]:
# arg
load_weights = False
plot_data = True 

import matplotlib
if not plot_data:
    matplotlib.use("Agg") # this is necessary when using plt without display (batch)
import matplotlib.pyplot as plt

In [ ]:
# infer signal (0), noise (1), or 1st cluster (2)?
extract_type = 2


# Number of channels (CNN channel = waveform channanels)
#nchannels = 1
nchannels = 2

if extract_type == 0:
  filename = f"denoising1D{nchannels}ch_tf2"
elif extract_type == 1:
  filename = f'noiseextraction1D{nchannels}ch_tf2'
else:
  filename = f'clustertiming1D{nchannels}ch_tf2' 

# Waveform has 1024 sample-points
npoints = 512 # 256 # number of sample-points to be used
scale = 1 #5
offset = 0.0 #0.05 # 50 mV

#signal_dataset_file = 'wf11600.pkl.gz'
##noise_dataset_file  = 'wf328469.pkl' #2018
#noise_dataset_file  = 'wf356990.pkl.gz' #2020
#signal_dataset_file = 'wf_spx13000.pkl.gz'
#noise_dataset_file  = 'wf_spx397695.pkl.gz' #2021
signal_dataset_file = 'wf_cdch13000.pkl.gz'
noise_dataset_file  = 'wf402042.pkl.gz' #2021
cluster_dataset_file = 'cls1st_cdch13000.pkl.gz' 

#### Hyper-parameters

In [ ]:
kernel_size = [9, 9, 9]

# basic hyper-parameters
params = {
    'optimizer':   'adam',
    'loss':        'mse',
    'metrics':     ['mae', 'mse'],
    'epochs':      150, # 20,
    'batch_size':  512, #256,
    'steps_per_execution': 100, # default 1, for TPU larger value may help
}
if extract_type == 0:
  params['loss'] = 'msle'
elif extract_type == 2:
  params['loss'] = 'binary_crossentropy'

# additional parameters
params2 = {
    'loss_type':           params['loss'],
    'conv_activation':     'relu',
    'output_activation':   'linear', #'sigmoid',
    'signal_dataset_file': signal_dataset_file,
    'noise_dataset_file':  noise_dataset_file,
    'File name':           filename + '.ipynb',
    'batch_size':          params['batch_size'],
    'npoints':             npoints,
    'nchannels':           nchannels,
    'nrows':               1,
    'scale':               scale,
    'offset':              offset,
    'nsublayers':          1,
    'nkernels':            [128, 64, 64,   64, 64, 128],
    'kernel_size':         kernel_size,
    'skip_connection':     [True, True, True],
}

## Prepare datasets
On Google Colb, data is loaded via Google Drive.
Files are supposed to be in `/content/drive/My Drive/ML/data`.

### Mount Google Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#data_dir = '/content/drive/My Drive/ML/data/'
#output_dir = '/content/drive/My Drive/ML/results/'

In [ ]:
## Here is a trick to mount Drive in other account
## First, run this column and click the URL shown in the error message.
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
data_dir = '/content/drive/MyDrive/ML/data/'
output_dir = '/content/drive/MyDrive/ML/results/'

### Load pickle files

In [ ]:
x_signal = pd.read_pickle(data_dir+signal_dataset_file).to_numpy()
x_noise = pd.read_pickle(data_dir+noise_dataset_file ).to_numpy()
if extract_type == 2:
  x_cluster = pd.read_pickle(data_dir+cluster_dataset_file ).to_numpy()

In [ ]:
nsamples = min(len(x_signal), len(x_noise)) 
nsamples = int(nsamples / nchannels) * nchannels

print(f'signal samples:{len(x_signal)}, noise samples:{len(x_noise)}, nsamples: {nsamples}')

if extract_type == 0:
  print('Extract signal')
  x_tobe_removed = x_noise[0:nsamples]
  x_tobe_extracted = x_signal[0:nsamples]
  x_true_signal = x_tobe_extracted
elif extract_type == 1:
  print('Extract noise')
  x_tobe_removed = x_signal[0:nsamples]
  x_tobe_extracted = x_noise[0:nsamples]
  x_true_signal = x_tobe_extracted
else:
  x_tobe_removed = x_noise[0:nsamples]
  x_tobe_extracted = x_cluster[0:nsamples]
  x_true_signal = x_signal[0:nsamples]

### Shape data in appropriate format with adding noise

In [ ]:
x_tobe_removed = x_tobe_removed.astype('float32')
x_tobe_removed = x_tobe_removed.T[-npoints:].T # keep last npoints
x_tobe_extracted = x_tobe_extracted.astype('float32')
x_tobe_extracted = x_tobe_extracted.T[-npoints:].T # keep last npoints
x_true_signal = x_true_signal.astype('float32')
x_true_signal = x_true_signal.T[-npoints:].T # keep last npoints

# Add noise
x_train_noisy = x_tobe_removed + x_true_signal

# Adjust scale and offset of waveforms
x_tobe_removed *= scale
x_tobe_removed += offset * scale
x_tobe_extracted *= scale # scale
x_tobe_extracted += offset * scale;
x_train_noisy *= scale # scale
x_train_noisy += offset * scale; # add offset
x_true_signal *= scale

## Values in [0,1]
#x_tobe_extracted = np.clip(x_tobe_extracted, 0, 1);
#x_train_noisy = np.clip(x_train_noisy, 0, 1);

# To match the input shape for Conv1D with n channels
x_tobe_removed = np.reshape(x_tobe_removed, (int(len(x_tobe_removed) / nchannels), nchannels, npoints)).transpose(0,2,1)
x_tobe_extracted = np.reshape(x_tobe_extracted, (int(len(x_tobe_extracted) / nchannels), nchannels, npoints)).transpose(0,2,1)
x_train_noisy = np.reshape(x_train_noisy, (int(len(x_train_noisy) / nchannels), nchannels, npoints)).transpose(0,2,1)
x_true_signal = np.reshape(x_true_signal, (int(len(x_true_signal) / nchannels), nchannels, npoints)).transpose(0,2,1)

print(x_tobe_removed.shape)

## Model

In [ ]:
# Add the following code anywhere in your machine learning file
# api_key and workspace are supposed to be set in .comet.config file,
# otherwise set here like Experiment(api_key="AAAXXX", workspace = "yyy", project_name="zzz")
# experiment = Experiment(project_name="wf_denoising")
experiment = Experiment(api_key="gBJn86Y1oAYKM2oxaoY0oV4Af", workspace="yskuchi", project_name="wf_denoising")

In [ ]:
experiment.log_parameters(params2)

### Setup TPU
This part seems tf version dependent and may be changed.

In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
  tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  tf.config.experimental_connect_to_cluster(tpu_cluster_resolver) # TF2.0の場合、ここを追加
  tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver) # TF2.0の場合、今後experimentialが取れる可能性がある    
  strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)  # ここも同様
  #model = tf.distribute.tpu.keras_to_tpu_model(model, strategy=strategy)

### Build model with functional API

In [ ]:
def build_model():
  input_img = Input(shape=(npoints,nchannels))
  conv1 = Conv1D(params2['nkernels'][0], kernel_size[0], padding='same', activation=params2['conv_activation'])(input_img)
  for i in range(1, params2['nsublayers']):
    conv1 = Conv1D(params2['nkernels'][0], kernel_size[0], padding='same', activation=params2['conv_activation'])(conv1)
  pool1 = MaxPooling1D(2, padding='same')(conv1)
  conv2 = Conv1D(params2['nkernels'][1], kernel_size[1], padding='same', activation=params2['conv_activation'])(pool1)
  for i in range(1, params2['nsublayers']):
    conv2 = Conv1D(params2['nkernels'][1], kernel_size[1], padding='same', activation=params2['conv_activation'])(conv2)
  pool2 = MaxPooling1D(2, padding='same')(conv2)
  conv3 = Conv1D(params2['nkernels'][2], kernel_size[2], padding='same', activation=params2['conv_activation'])(pool2)
  for i in range(1, params2['nsublayers']):
    conv3 = Conv1D(params2['nkernels'][2], kernel_size[2], padding='same', activation=params2['conv_activation'])(conv3)
  encoded = MaxPooling1D(2, padding='same')(conv3)

  conv4 = Conv1D(params2['nkernels'][3], kernel_size[2], padding='same', activation=params2['conv_activation'])(encoded)
  for i in range(1, params2['nsublayers']):
    conv4 = Conv1D(params2['nkernels'][3], kernel_size[2], padding='same', activation=params2['conv_activation'])(conv4)
  up5 = UpSampling1D(2)(conv4)
  if params2['skip_connection'][2]:
    up5 = Concatenate()([up5, conv3])
  conv5 = Conv1D(params2['nkernels'][4], kernel_size[1], padding='same', activation=params2['conv_activation'])(up5)
  for i in range(1, params2['nsublayers']):
    conv5 = Conv1D(params2['nkernels'][4], kernel_size[1], padding='same', activation=params2['conv_activation'])(conv5)
  up6 = UpSampling1D(2)(conv5)
  if params2['skip_connection'][1]:
    up6 = Concatenate()([up6, conv2])
  conv6 = Conv1D(params2['nkernels'][5], kernel_size[0], padding='same', activation=params2['conv_activation'])(up6)
  for i in range(1, params2['nsublayers']):
    conv6 = Conv1D(params2['nkernels'][5], kernel_size[0], padding='same', activation=params2['conv_activation'])(conv6)
  up7 = UpSampling1D(2)(conv6)
  if params2['skip_connection'][0]:
    up7 = Concatenate()([up7, conv1])
  for i in range(params2['nsublayers'] - 1):
    up7 = Conv1D(1, kernel_size[0], padding='same', activation=params2['conv_activation'])(up7)
  decoded = Conv1D(nchannels, kernel_size[0], padding='same', activation=params2['output_activation'])(up7)

  autoencoder = Model(inputs=input_img, outputs=decoded)

  autoencoder.compile(optimizer=params['optimizer'], 
                      loss=params['loss'], 
                      metrics=params['metrics'],
                      steps_per_execution=params['steps_per_execution']) 
  autoencoder.summary()
  return autoencoder

In [ ]:
try:
  strategy
  with strategy.scope():
    autoencoder = build_model()
except NameError:
  autoencoder = build_model()

## Fit

On Google Colb, the results (trained model) are saved in Google Drive. Files are supposed to be in /content/drive/My Drive/ML/results.

In [ ]:
history=[]
if not load_weights:

    # Callback for model checkpoints
    checkpoint = ModelCheckpoint(
        filepath = output_dir + filename + "-{epoch:02d}.h5",
        save_best_only=True,
        save_weight_only=False)
    
    # 'labels' are the pictures themselves
    hist = autoencoder.fit(x_train_noisy, x_tobe_extracted,
                           epochs=params['epochs'],
                           batch_size=params['batch_size'],
                           shuffle=True,
                           validation_split=0.1)#,
                           #callbacks=[checkpoint])


    # Save history
    with open(output_dir + filename + '_hist.json', 'w') as f:
        json.dump(hist.history, f)
    history = hist.history
        
    # Save the weights
    autoencoder.save_weights(output_dir + filename + '_weights.h5')
else:
    # Load weights
    autoencoder.load_weights(f'{output_dir}{filename}_weights.h5')

    # Load history
    with open(f'{output_dir}{filename}_hist.json', 'r') as f:
        history = json.load(f)

autoencoder.save(output_dir + filename + '.h5', include_optimizer=False)
        
# Plot training history 
#plt.plot(history['loss'], linewidth=3, label='train')
#plt.plot(history['val_loss'], linewidth=3, label='valid')
plt.plot(history['mae'], linewidth=3, label='train')
plt.plot(history['val_mae'], linewidth=3, label='valid')
plt.grid()
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
if params['metrics'] == 'mse':
  plt.ylim(1e-6 * scale, 0.1e-3 * scale) #mse
else:
  plt.ylim(0.5e-3 * scale, 0.5e-2 * scale) #mae
plt.show()

## Test

In [ ]:
x_test_tobe_removed = x_tobe_removed[0:]
x_test_tobe_extracted = x_tobe_extracted[0:]
x_test_noisy = x_train_noisy[0:]
decoded_imgs = autoencoder.predict(x_test_noisy)

# revert scale and offset
x_test_tobe_removed -= scale * offset
x_test_tobe_removed /= scale
x_test_tobe_extracted -= scale * offset
x_test_tobe_extracted /= scale
x_test_noisy -= scale * offset
x_test_noisy /= scale
decoded_imgs -= scale * offset
decoded_imgs /= scale
#x_subtracted = x_test_tobe_extracted - decoded_imgs

print(x_test_tobe_extracted.shape)
print(x_test_noisy.shape)

x_test_tobe_removed = x_test_tobe_removed.transpose(0,2,1)
x_test_tobe_removed = np.reshape(x_test_tobe_removed, (len(x_test_tobe_removed) * nchannels, npoints))
x_test_tobe_extracted = x_test_tobe_extracted.transpose(0,2,1)
x_test_tobe_extracted = np.reshape(x_test_tobe_extracted, (len(x_test_tobe_extracted) * nchannels, npoints))
x_test_noisy = x_test_noisy.transpose(0,2,1)
x_test_noisy = np.reshape(x_test_noisy, (len(x_test_noisy) * nchannels, npoints))
decoded_imgs = decoded_imgs.transpose(0,2,1)
decoded_imgs = np.reshape(decoded_imgs, (len(decoded_imgs) * nchannels, npoints))

#x_subtracted = x_subtracted.transpose(0,2,1)
#x_subtracted = np.reshape(x_subtracted, (len(x_subtracted) * nchannels, npoints))
x_residual = x_test_tobe_extracted - decoded_imgs
x_decoded_signal = decoded_imgs
if extract_type == 1:
  x_decoded_signal = x_test_noisy - decoded_imgs

In [ ]:
# How many waveforms to be displayed
n = 2
start = 16
fig = plt.figure(figsize=(20, 6 * n))
j = 0
for i in range(start, start + n):
  ax = fig.add_subplot(n, 1, j+1)
  #ax.plot(x_test_tobe_removed[i], label="original")
  ax.plot(x_test_noisy[i], label="noisy", color='gray')
  if extract_type == 1:
    ax.plot(x_test_tobe_extracted[i], label="noise", color='green')
    ax.plot(x_test_tobe_removed[i], label="signal")
    ax.plot(x_decoded_signal[i], label="extracted signal")
  else:
    ax.plot(x_test_tobe_extracted[i], label="signal", color='green')
  ax.plot(decoded_imgs[i], label="decoded", color='magenta')
  ax.plot(x_residual[i], label="residual")
  ax.legend()
  j += 1

In [ ]:
# Send this plot to comet
experiment.log_figure(figure=fig)

In [ ]:
experiment.end()

In [ ]:
if plot_data:
    plt.show()